<a href="https://colab.research.google.com/github/nrutyadoshi/ML/blob/master/J013_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import pandas as pd
import numpy as np

In [0]:
upload = files.upload()

In [0]:
df = pd.read_csv("Credit.csv")

In [0]:
df.drop(["Unnamed: 0"],1,inplace=True)

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
scaler = MinMaxScaler()

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
encoder = LabelEncoder()
df["Gender"] = encoder.fit_transform(df["Gender"].astype("str"))
df["Student"] = encoder.fit_transform(df["Student"].astype("str"))
df["Married"] = encoder.fit_transform(df["Married"].astype("str"))
df["Cards"] = encoder.fit_transform(df["Cards"].astype("str"))
#df["Cards"] = pd.Categorical(df.Cards)

In [0]:
df = df.join(pd.get_dummies(df["Ethnicity"])) # One hot encoding

In [0]:
df.drop(["Ethnicity","Caucasian"],1,inplace=True)

In [0]:
df.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Balance,African American,Asian
0,14.891,3606,283,1,34,11,0,0,1,333,0,0
1,106.025,6645,483,2,82,15,1,1,1,903,0,1
2,104.593,7075,514,3,71,11,0,0,0,580,0,1
3,148.924,9504,681,2,36,11,1,0,0,964,0,1
4,55.882,4897,357,1,68,16,0,0,1,331,0,0


In [0]:
X = df.drop(["Balance"],1)
Y = df["Balance"]

In [0]:
from sklearn.model_selection import train_test_split as tts

In [0]:
X_train, X_test, Y_train, Y_test = tts(X,Y,test_size=0.3,random_state=42)

In [0]:
train = pd.concat([X_train, Y_train],1)
test = pd.concat([X_test, Y_test],1)

In [0]:
train.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,African American,Asian,Balance
157,56.256,5521,406,1,72,16,1,1,1,0,0,1020
109,13.561,3261,279,4,37,19,0,0,1,0,1,297
17,36.496,4378,339,2,69,15,1,0,1,0,1,368
347,160.231,10748,754,1,69,17,0,0,0,0,0,1192
24,10.742,1757,156,2,57,15,1,0,0,0,0,0


In [0]:
train_scale = pd.DataFrame(scaler.fit_transform(train),columns = train.columns)
test_scale = pd.DataFrame(scaler.fit_transform(test),columns = test.columns)

In [0]:
train_scale.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,African American,Asian,Balance
0,0.260393,0.357329,0.344710,0.125,0.648649,0.714286,1.0,1.0,1.0,0.0,0.0,0.510255
1,0.018193,0.184255,0.200228,0.500,0.175676,0.928571,0.0,0.0,1.0,0.0,1.0,0.148574
2,0.148298,0.269796,0.268487,0.250,0.608108,0.642857,1.0,0.0,1.0,0.0,1.0,0.184092
3,0.850221,0.757620,0.740614,0.125,0.608108,0.785714,0.0,0.0,0.0,0.0,0.0,0.596298
4,0.002201,0.069076,0.060296,0.250,0.445946,0.642857,1.0,0.0,0.0,0.0,0.0,0.000000


In [0]:
X_train = train_scale.drop(["Balance"],1)
Y_train = train_scale["Balance"]
X_test = test_scale.drop(["Balance"],1)
Y_test = test_scale["Balance"]

In [0]:
X_train.shape

(280, 11)

In [0]:
import statsmodels.api as sm

In [0]:
cols = X_train.columns.to_list()
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X_train[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(Y_train,X_train).fit()
    #p = pd.Series(model.pvalues.values[0:],index = cols)      
    #pmax = max(p)
    #feature_with_p_max = p.idxmax() # returns index of max p value
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [0]:
model.summary()

In [0]:
X = df_final[selected_features_BE]

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [0]:
reg = LinearRegression()
linear = reg.fit(X_train,Y_train)
Y_pred = linear.predict(X_test)

In [0]:
print(r2_score(Y_test,Y_pred))

0.9512075865767012


In [0]:
linear.intercept_

-266.92177130208995

In [0]:
linear.coef_

array([   9.72791749,   -9.72791749, -208.7374536 ,  208.7374536 ,
          0.66143092,   -0.66143092,   -9.39694162,    3.84511944,
          5.55182218,   -7.54192364,    0.22185   ,    0.60851245,
         21.90990342,   -0.55031829])

In [0]:
from sklearn.neighbors import KNeighborsRegressor

In [0]:
knn = KNeighborsRegressor()
knn_reg = knn.fit(X_train,Y_train)
Y_pred2 = knn_reg.predict(X_test)

In [0]:
print(r2_score(Y_test,Y_pred2))

0.5168309146768737


In [0]:
from sklearn.linear_model import RANSACRegressor

In [0]:
ransac = RANSACRegressor()
ran = ransac.fit(X_train,Y_train)
Y_pred1 = ran.predict(X_test)

In [0]:
print(r2_score(Y_test,Y_pred1))

0.9514110035326987


In [0]:
from sklearn.linear_model import ElasticNet

In [0]:
elastic=ElasticNet(normalize=True)
search=GridSearchCV(estimator=elastic,param_grid={'alpha':np.logspace(-5,2,8),'l1_ratio':[.2,.4,.6,.8]},scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
import numpy as np
params = {"n_neighbors":np.arange(2,14,1), 
         "metric":["euclidean", "minkowski", "jaccard", "cosine"]}

In [0]:
knn_cv = GridSearchCV(elastic, param_grid=params, cv = 10)

In [0]:
from sklearn.linear_model import ElasticNetCV,ElasticNet # ERROR
from sklearn.model_selection import GridSearchCV

In [0]:
elastinet = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1],cv=5)
parametersGrid = {"max_iter": [1, 5, 10],
                      "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0, 1.0, 0.1)}

In [0]:
eNet = ElasticNet()
grid = GridSearchCV(eNet, parametersGrid, scoring='r2', cv=10)
grid.fit(X_train,y_train)
grid.best_params_